<div style="text-align:center;">
  <img src="custom/molssi_main_horizontal.png" style="display: block; margin: 0 auto; max-height:200px;">
</div>

# Creating a Solubility Model

<strong>Author(s):</strong> Jessica A. Nash, Sina Mostafanejand, The Molecular Sciences Software Institute

In this notebook, your task is to build a model for predicting the solubility of small molecules. We will use a dataset from a [2004 paper by John S. Delaney](https://pubs.acs.org/doi/10.1021/ci034243x) and aim to emulate the fitting of the ESOL (Estimated Solubility) model published in the paper.
The Delaney model is a multi-linear regression model where small molecule solubility is predicted based on four molecular descriptors.

## Molecular Solubility
The solubility of a molecule is a critical property in medicinal chemistry, drug discovery, and agrochemistry as it determines the bioavailability of a drug or pesticide.
**Partition coefficient**, $P$, measures the propensity of a neutral compound to dissolve in a mixture of two immiscible solvents, often water and octanol. In simple terms, it determines how much of a solute dissolves in the aqueous phase versus the organic portion.

The partition coefficient is defined as:

$$
P = \frac{[Solute]_{\text{organic}}}{[Solute]_{\text{aqueous}}}     
$$

where $[Solute]_{\text{organic}}$ and $[Solute]_{\text{aqueous}}$ are the concentrations of the solute in the organic and aqueous phases, respectively.
Instead of $P$, we often use the logarithm of the partition coefficient, $\log P$, which is more convenient to work with. A negative value of $\log P$ indicates that the solute is more soluble in water, while a positive value indicates that the solute is more soluble in the organic phase. When $\log P$ is close to zero, the solute is equally soluble in both phases. Although $\log P$ is a constant value, its magnitude is dependent on the measurement conditions and the choice of the organic solvent.

<div class="alert alert-block alert-warning"> 
    <h2>Check Your Understanding</h2>
    What does logP value of 1 mean?
</div>



## Molecular Descriptors for Solubility Fitting

The original ESOL model fits solubility as a multi-linear regression model of the following molecular descriptors.

1. clogP
2. Molecular Weight
3. Number of rotatable bonds
4. Aromatic proportion


<div class="alert alert-block alert-success"> 
<strong>Calculating Descriptors</strong>

One task that may take you some time is figuring out how to calculate these descriptors. 
When in doubt, you may decided to use Google. The RDKit documentation is always a good source for information.

For the first descriptor, `clogP`, you should use the [rdkit Crippen module](http://rdkit.org/docs/source/rdkit.Chem.Crippen.html).

You can import it using

```python
from rdkit.Chem import Crippen
```

</div>

The model published in the original paper is the following based on these four descriptors

$$
\text{Log}(S_W) = 0.16 - 0.63 \, \text{clogP} - 0.0062 \, \text{MWT} +  0.066 \, \text{RB} - 0.74 \, \text{AP}
$$


Note that our dataset comes from the Supplemental Information of the original paper, and is only for a subset of the compounds considered in the original model.

## Tasks

The Supplemental Information from the paper is in the file `data/ESOL_supplemental.csv`.

Your tasks are to:

1. Load the data using `pandas`
2. Calculate molecular descriptors using RDKit.
3. Examine relationships using visualization tools - built in Pandas plotting or Seaborn.
4. Select your data and do a train-test split.
5. Train your model
6. Evaluate your model.


<div class="alert alert-block alert-success"> 
<strong>Fitting Multi-Linear Regression</strong>

When performing multi-linear regression with SciKit Learn, you will follow a similar procedure to the one we followed for linear regression
except that when you select your columns for your X values, you should select multiple columns.

</div>

## Task 1 - Load data using pandas

In [ ]:
import pandas as pd
from rdkit.Chem import PandasTools

# Ensure molecules are rendered in the notebook
PandasTools.RenderImagesInAllDataFrames(images=True)

df = pd.read_csv("data/ESOL_supplemental.csv")
df.head()

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df,'SMILES','Molecule',includeFingerprints=True)

df.head(3)

In [ ]:
df.info()

# Task 2
Calculate molecular descriptors using RDKit.

1. clogP
2. Molecular Weight
3. Number of rotatable bonds
4. Aromatic proportion

In [ ]:
from rdkit import Chem
from rdkit.Chem import Crippen
from rdkit.Chem import Descriptors

In [ ]:
df["clogP"] = df["Molecule"].apply(Crippen.MolLogP)
df["MolWt"] = df["Molecule"].apply(Descriptors.MolWt)
df["NumRotatable"] = df["Molecule"].apply(Descriptors.NumRotatableBonds)
df["AromaticProportion"] = df["Molecule"].apply(lambda x: len(x.GetAromaticAtoms()) / x.GetNumAtoms())

In [ ]:
df.head()

## Task 3 - Examine relationships using visualization tools - built in Pandas plotting or Seaborn.

In [ ]:
df.hist(figsize=(8,8), edgecolor='black', grid=False)

In [ ]:
corr = df.corr(numeric_only=True)
corr

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")

## Task 4 - Select your data and do a train-test split.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


X = df[["clogP", "MolWt", "NumRotatable", "AromaticProportion"]]
Y = df[["measured log(solubility:mol/L)"]]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

ttt_model = LinearRegression()
ttt_model.fit(X_train, Y_train)

In [ ]:
y_pred = ttt_model.predict(X_test)

# Print the model coefficients
print(f"Model coefficient (slope): {ttt_model.coef_}")
print(f"Model intercept: {ttt_model.intercept_[0]}")

r2_score(y_pred, Y_test)

In [ ]:
# Plot the results
y_values = Y.values
y_predict_all = ttt_model.predict(X)

plt.scatter(y_values, y_predict_all)

plt.plot([min(y_values), max(y_values)], [min(y_values), max(y_values)], 'r--', label='Ideal line (y=x)')
plt.xlabel('Actual Solubility')
plt.ylabel('Predicted Solubility')
plt.legend()
plt.grid(True)
plt.axis('equal')